In [ ]:
import numpy as np
import pandas as pd
import sentence_transformers
import nltk
import gensim
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import bertopic
import spacy
from transformers import pipeline

In [ ]:
df_news = pd.read_json('data/cc_gw_news_blogs_2021-10-01_2021-10-31.json')

# Text cleaning and searching

How many of the headlines mention Justin Trudeau?

In [ ]:
## We'll consider a headline as relevant if it mentions just his surname.
count = 0
for t in df_news.title:
    if 'trudeau' in t.lower():
        count += 1
print(f'{count} titles mention Trudeau.')

How many of the headlines mention one of the G7 countries?

In [ ]:
## This query is not an exhaustive list - you could do more to match the UK and US.
query = r'(Canada|Italy|France|Germany|Japan|United Kingdom|UK|United States|US|)'
count = 0
for t in df_news.title:
    if np.any([m != '' for m in re.findall(query,t)]):
        count += 1
print(f'{count} titles mention a G7 country.')

# Vectorisation and embeddings

Find the five most similar headline among the first 1000 headlines to the provided text.

In [ ]:
sentence_model = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')

ref_text = ['President Biden visited other G7 leaders in London']
sents = ref_text + [t for t in df_news.title[:1000]]
embs = sentence_model.encode(sents,show_progress_bar=True)

## We can calculate pairwise similarity easily.
sim = cosine_similarity(embs)
## The first row gives the similarity scores we care about. Find the maximal elements.
matches = np.argsort(sim[0])
for i in matches[-6:-1]:
    print(sents[i],sim[0,i])

Which of the first 1000 articles uses *climate* the most?

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
X_1k = vectorizer.fit_transform([t for t in df_news.body[:1000]])

## We need to get the feature names from vectorizer to find out which column of X_1k to check.
clim_col = list(vectorizer.get_feature_names_out()).index('climate')

max_doc = np.argmax(X_1k[:,clim_col])
print(f'The {max_doc}th article uses climate {X_1k[max_doc,clim_col]} times')

# Topic modelling

Fit a topic model over the first 1000 article bodies using the default BERTopic settings. Find the list of row numbers in the dataset that are about the president of Zimbabwe according to the topic model.

In [ ]:
topic_model = bertopic.BERTopic()
topics, probs = topic_model.fit_transform(df_news.body[:1000])
df_topics = topic_model.get_topic_info()
df_topics

In [ ]:
## Note the that the topic label here is dependent on a particular iteration of the topic model (unless you fix the random state)
## The topic number should be around 25 in most cases however.
print(f'Row ids corresponding to the president of Zimbabwe are: {[i for i,t in enumerate(topics) if t == 25]}')

The topics and probabilities assigned by BERTopic correspond to the best fit. Using the probabilities return for each text assigned to the topics in your previous model, find the topics with the highest and lowest mean probability. Do these values tell you anything about the topics highlighted?

In [ ]:
## First we convert topics and probs to a more convenient format.
## This gives a list of probabilities for each topic label.
topic_probs = {}
for t,p in zip(topics,probs):
        if not topic_probs.get(t):
            topic_probs[t] = []
        topic_probs[t].append(p)

## Next we average them and record the maximum and minimum we've seen.
max_avg_topic = -2
min_avg_topic = -2
max_avg = 0
min_avg = 1
for t in topic_probs:
    avg = np.mean(topic_probs[t])
    if avg > max_avg:
        max_avg_topic = t
        max_avg = avg
    if avg < min_avg:
        min_avg_topic = t
        min_avg = avg

## This is almost certainly going to be topic -1 with probability 0 - BERTopic doesn't record probabilities for the junk topic.
## This is because they don't really make sense.
print(f'The topic with lowest average probability is {min_avg_topic} with probability {min_avg}.')
## This is probably one of the smaller topics with a very high probability. Be careful of overfitting to duplicate or
## very similar texts in your dataset causing the model to converge anomalously.
print(f'The topic with highest average probability is {max_avg_topic} with probability {max_avg}.')

# Text classification

Find the 10 most common entities across the first 1000 articles. Hint: you may find the `collections` library useful.

In [ ]:
import collections
import tqdm  ## This displays a progress bar to track efficiency.

nlp = spacy.load("en_core_web_sm")

ents = []
docs = []
for i, row in tqdm.tqdm(df_news[:1000].iterrows()):
    doc = nlp(row.body)
    docs.append(doc)  ## Saving these to avoid repeating the calculation.
    ents += [e.text for e in doc.ents]  ## Note that taking the text is required, otherwise the entities will not be matched.
ent_counts = collections.Counter(ents)
print(f'The ten most common entities are {ent_counts.most_common(10)}.')

Different part of speech tags will appear with different frequencies. Find the articles among the first 1000 articles that that have the highest and lowest proportion of proper nouns in their bodies.

In [ ]:
max_prop = 0
min_prop = 1
max_prop_id = -1
min_prop_id = -1

for i,doc in enumerate(docs):
    pns = 0
    for t in doc:
        if t.pos_ == 'PROPN':
            pns += 1
    prop = pns/len(doc)
    
    if prop > max_prop:
        max_prop = prop
        max_prop_id = i
    if prop < min_prop:
        min_prop = prop
        min_prop_id = i

print(f'Article {min_prop_id} has the lowest proportion of proper nouns: {min_prop}.')
print(f'Article {max_prop_id} has the lowest proportion of proper nouns: {max_prop}.') 

News media is sometimes criticised as focused on negative stories. How are the sentiment labels and class probabilities split over the first 1000 headlines? Does this trend tell you anything about the classes or your chosen model?

In [ ]:
import matplotlib.pyplot as plt
## The example model used earlier in these notes doesn't give a range of sentiment scores and rarely assigns neutral.
## Instead it gives a positive or negative label with a probability of being positive or negative.
## We should use a different model - the Cardiff NLP group is a good source.

sentiment_analysis = pipeline("sentiment-analysis",model="cardiffnlp/twitter-roberta-base-sentiment-latest")
sents = [sentiment_analysis(t)[0] for t in df_news[:1000].title]

## Each score is a dict, label and probability. We need to split them into pos, neu, neg.
pos,neu,neg = [],[],[]
for s in sents:
    if s['label'] == 'positive':
        pos.append(s['score'])
    elif s['label'] == 'neutral':
        neu.append(s['score'])
    else:
        neg.append(s['score'])

print(f'Proportion of positive headlines: {len(pos)/1000}')
print(f'Proportion of neutral headlines: {len(neu)/1000}')
print(f'Proportion of negative headlines: {len(neg)/1000}')

plt.hist(pos,label='Positive',density=True)
plt.hist(neu,label='Neutral',density=True)
plt.hist(neg,label='Negative',density=True)
plt.legend()
plt.show()

## We see that most of the headlines are reported as neutral, with slightly more negative than positive.
## The model may be less certain of the negative labels (some slight skew in the distribution of probabilities).
## This may be an indication that it is conflating 'neutral' and 'not sure', but this is not conclusive.
## Analysis based on neutral labels should therefore be considered carefully to ensure there are no confounding factors.